In [1]:
import os
import random
from itertools import combinations
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

In [2]:
repo_dir = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
labelled_data_dir = f"{repo_dir}/data/labelled/"

In [43]:
selected_columns = ['example_id', 'comment_for_evaluation','toxicity', 'severe_toxicity', 'identity_attack',
       'insult', 'profanity', 'threat']

all_labelled_df = []
for file_name in os.listdir(labelled_data_dir):
    labelled_df = pd.read_csv(f"{labelled_data_dir}{file_name}")[selected_columns]
    labelled_df['rater_id'] = file_name.split("_")[0]
    all_labelled_df.append(labelled_df)

In [44]:
labelled_df = pd.concat(all_labelled_df)

In [10]:
n_raters_lookup = labelled_df .groupby("example_id").rater_id.nunique().reset_index()

In [9]:
labelled_df['example_id']

,example_id,toxicity,severe_toxicity,identity_attack,insult,profanity,threat,rater_id
0,102693,0.0,0.0,0.0,0.0,0.0,0.0,yj2369
1,115299,0.0,0.0,0.0,0.0,0.0,0.0,yj2369
2,120753,0.0,0.0,0.0,0.0,0.0,0.0,yj2369
3,128106,0.0,0.0,0.0,0.0,0.0,0.0,yj2369
4,133758,0.0,0.0,0.0,0.0,0.0,1.0,yj2369
...,...,...,...,...,...,...,...,...
255,84775,0.0,0.0,0.0,0.0,0.0,0.0,yj2369
256,85744,0.0,0.0,0.0,0.0,0.0,0.0,yj2369
257,90036,0.0,0.0,1.0,0.0,0.0,0.0,yj2369
258,93060,0.0,0.0,1.0,0.0,0.0,0.0,yj2369


In [13]:
example_ids_for_interrater = n_raters_lookup[n_raters_lookup['rater_id'] == 2]['example_id'].values

In [18]:
rater_ids = ['ac4119', 'gm2858', 'yj2369','yp2201']
num_raters = len(rater_ids)

In [16]:
labelled_df_1 = labelled_df[labelled_df['example_id'].isin(example_ids_for_interrater)]

In [23]:
interrater_assessments_df = pd.melt(labelled_df_1,
                                    id_vars=['example_id','rater_id'], 
                                    value_vars=labels)\
                              .rename(columns={'value':'label'})

In [20]:
labels = ['toxicity', 'severe_toxicity', 'identity_attack', 'insult', 'profanity', 'threat']

In [25]:

interrater_assessments_df = pd.pivot(interrater_assessments_df , 
                                     index=['variable', 'example_id'], 
                                     columns='rater_id', values='label').fillna(0).reset_index()

In [30]:
interrater_assessments_df.columns = ['variable', 'example_id', 0, 1, 2, 3]

In [27]:
all_rater_combinations = list(combinations(range(num_raters), 2))

In [28]:
all_rater_combinations

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

In [36]:
spearmanr_reliability = []
for rater_x, rater_y in all_rater_combinations:
    r_coef, p_value = spearmanr(interrater_assessments_df[rater_x].values, interrater_assessments_df[rater_y].values)
    data = {"rater_pair": (rater_x, rater_y), "r_coef": r_coef, "p_value": p_value, "spearmanr_reliability_type":"overall"}
    spearmanr_reliability.append(data)

    for variable in interrater_assessments_df['variable'].unique():
        print(variable)
        rater_x_values = interrater_assessments_df[interrater_assessments_df['variable']==variable][rater_x].values
        rater_y_values = interrater_assessments_df[interrater_assessments_df['variable']==variable][rater_y].values
        var_r_coef, var_p_value = spearmanr(rater_x_values, rater_y_values)
        data = {"rater_pair": (rater_x, rater_y), "r_coef": var_r_coef, "p_value": var_p_value, 
                "spearmanr_reliability_type":f"by_var_{variable}"}
        spearmanr_reliability.append(data)

identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity
identity_attack
insult
profanity
severe_toxicity
threat
toxicity


/Users/ameliachu/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [37]:
spearmanr_reliability_df = pd.DataFrame(spearmanr_reliability)[['spearmanr_reliability_type', 'rater_pair', 'r_coef','p_value']].fillna(1)

In [38]:
spearmanr_reliability_df

,spearmanr_reliability_type,rater_pair,r_coef,p_value
0,overall,"(0, 1)",0.246898,2.024133e-41
1,by_var_identity_attack,"(0, 1)",0.191614,2.283704e-05
2,by_var_insult,"(0, 1)",0.108952,1.671557e-02
3,by_var_profanity,"(0, 1)",0.216531,1.598336e-06
4,by_var_severe_toxicity,"(0, 1)",1.000000,1.000000e+00
5,by_var_threat,"(0, 1)",1.000000,1.000000e+00
6,by_var_toxicity,"(0, 1)",0.272045,1.263278e-09
7,overall,"(0, 2)",0.208401,9.648073e-30
8,by_var_identity_attack,"(0, 2)",-0.013301,7.708451e-01
9,by_var_insult,"(0, 2)",0.189794,2.738428e-05


In [39]:
spearmanr_reliability_df.groupby('spearmanr_reliability_type').agg({'r_coef':np.mean})

,r_coef
spearmanr_reliability_type,
by_var_identity_attack,0.099962
by_var_insult,0.112610
by_var_profanity,0.163699
by_var_severe_toxicity,0.831732
by_var_threat,1.000000
by_var_toxicity,0.143832
overall,0.199455


In [45]:
spearmanr_reliability_df.groupby('rater_pair').agg({'r_coef':np.mean})

,r_coef
rater_pair,
"(0, 1)",0.433720
"(0, 2)",0.241584
"(0, 3)",0.313627
"(1, 2)",0.328842
"(1, 3)",0.485378
"(2, 3)",0.383669


In [48]:
profane_ids= set(labelled_df[labelled_df['profanity']==1]['example_id'].values)

In [63]:
x = labelled_df[(labelled_df['example_id'].isin(profane_disagreements )) & (labelled_df['example_id'].isin(example_ids_for_interrater))].drop_

In [60]:
labelled_df[labelled_df['example_id']==616547]

,example_id,comment_for_evaluation,toxicity,severe_toxicity,identity_attack,insult,profanity,threat,rater_id
182,616547,Yesterday rally's gain wiped out by 75% as of ...,0.0,0.0,0.0,0.0,1.0,0.0,yp2201
236,616547,Yesterday rally's gain wiped out by 75% as of ...,0.0,0.0,0.0,0.0,1.0,0.0,yj2369


In [70]:
y = x[['example_id','profanity']].drop_duplicates().groupby("example_id").profanity.nunique().reset_index()
y[y['']]

In [75]:
profane_disagreements = y[y['profanity'] == 2]['example_id'].values

In [77]:
x = labelled_df[(labelled_df['example_id'].isin(profane_disagreements))]

In [83]:
x.sort_values('example_id').to_csv(f"{repo_dir}/data/profane_disagreements.csv", index=False)

In [ ]:
 🍆 (when referring to genitals), moron, 
    
    damn,
    retard
    bitch
    "F's in the chat" vs. F
    piss
    ass